In [1]:
#carregamento de bibliotecas

from ucimlrepo import fetch_ucirepo 
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn import metrics
import time
from six import StringIO
from IPython.display import Image  
import pydotplus
import import_ipynb
from controllers.utils.utils import mostrar_dataframe_interativo
print("bibliotecas carregadas")


bibliotecas carregadas


In [2]:
#configuracoes iniciais

%load_ext autoreload
%autoreload 2
pd.set_option('display.max_columns', None)
pd.options.display.float_format = '{:.6f}'.format


In [3]:
#carregamendo dos dados

df = pd.read_csv("data.csv")
print(f"dados carregados: {df['cid'].count()} registros")


dados carregados: 2139 registros


In [4]:
# Embaralhando as linhas do DataFrame

df = df.sample(frac=1, random_state=None).reset_index(drop=True)
mostrar_dataframe_interativo(df)


IntSlider(value=10, description='Linhas até:', max=2139, min=10, step=5)

IntSlider(value=10, description='Linhas por vez:', max=2139, min=5, step=5)

Output()

In [5]:
# Extratificação 

X = df.drop(columns=['cid'])  # Variáveis independentes (todas as colunas menos a 'cid')
y = df['cid']  # Variável dependente (alvo)
print("Variaveis independentes")
mostrar_dataframe_interativo(X)
print()
print("Variavel dependente (target)")
mostrar_dataframe_interativo(y)


Variaveis independentes


IntSlider(value=10, description='Linhas até:', max=2139, min=10, step=5)

IntSlider(value=10, description='Linhas por vez:', max=2139, min=5, step=5)

Output()


Variavel dependente (target)


IntSlider(value=10, description='Linhas até:', max=2139, min=10, step=5)

IntSlider(value=10, description='Linhas por vez:', max=2139, min=5, step=5)

Output()

In [6]:
# Contagem das classes antes da divisão

print("Distribuição original das classes em 'cid':")
print(y.value_counts(normalize=True))  # Proporção das classes antes da divisão
print()


Distribuição original das classes em 'cid':
cid
0   0.756428
1   0.243572
Name: proportion, dtype: float64



In [7]:
# Primeiro, dividir em treino (50%) e restante (50% para validação + teste)

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.5, random_state=42, stratify=y)


In [8]:
# Contagem das classes no conjunto de treino

print("Distribuição das classes no conjunto de treino (X_train):")
print(y_train.value_counts(normalize=True))  # Proporção das classes no treino
print()


Distribuição das classes no conjunto de treino (X_train):
cid
0   0.756782
1   0.243218
Name: proportion, dtype: float64



In [9]:
# Segunda divisão: validação (25%) e teste (25%)

X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)


In [10]:
# Contagem das classes nos conjuntos de validação e teste

print("Distribuição das classes no conjunto de validação (X_val):")
print(y_val.value_counts(normalize=True))  # Proporção das classes na validação
print()

print("Distribuição das classes no conjunto de teste (X_test):")
print(y_test.value_counts(normalize=True))  # Proporção das classes no teste
print()


Distribuição das classes no conjunto de validação (X_val):
cid
0   0.755140
1   0.244860
Name: proportion, dtype: float64

Distribuição das classes no conjunto de teste (X_test):
cid
0   0.757009
1   0.242991
Name: proportion, dtype: float64



In [41]:
def update_results(results,algoritm,iteration,y_val, classifier):
    classifier.fit(X_train, y_train)
    y_pred = classifier.predict(X_val)  # Usando X_val e y_val para validação
    iteration['accuracy'] = metrics.accuracy_score(y_val, y_pred)
    iteration['confusion_matrix'] = metrics.confusion_matrix(y_val, y_pred).tolist()
    iteration['elapsed_time'] = time.time() - iteration['start_time']
    
    results[algoritm]['iterations'].append(iteration)

    # Atualizar melhores e piores índices e valores de acurácia e tempo
    if not results[algoritm]['best_accuracy'] or iteration['accuracy'] > results[algoritm]['best_accuracy']['accuracy']:
        results[algoritm]['best_accuracy'] = iteration

    if not results[algoritm]['worst_accuracy'] or iteration['accuracy'] < results[algoritm]['worst_accuracy']['accuracy']:
        results[algoritm]['worst_accuracy'] = iteration
        
    if not results['best_metrics']['best_accuracy'] or results['best_metrics']['best_accuracy']['accuracy'] < iteration['accuracy']: 
        results['best_metrics']['best_accuracy'] = dict(iteration)
        results['best_metrics']['best_accuracy']['algoritm'] = algoritm
    
    if not results['worst_metrics']['worst_accuracy'] or results['worst_metrics']['worst_accuracy']['accuracy'] > iteration['accuracy']: 
        results['worst_metrics']['worst_accuracy'] = dict(iteration)
        results['worst_metrics']['worst_accuracy']['algoritm'] = algoritm

    if not results[algoritm]['best_time'] or iteration['elapsed_time'] < results[algoritm]['best_time']['elapsed_time']:
        results[algoritm]['best_time'] = iteration

    if not results[algoritm]['worst_time'] or iteration['elapsed_time'] > results[algoritm]['worst_time']['elapsed_time']:
        results[algoritm]['worst_time'] = iteration
        
    if not results['best_metrics']['best_time'] or results['best_metrics']['best_time']['elapsed_time'] > iteration['elapsed_time']: 
        results['best_metrics']['best_time'] = dict(iteration)
        results['best_metrics']['best_time']['algoritm'] = algoritm
    
    if not results['worst_metrics']['worst_time'] or results['worst_metrics']['worst_time']['elapsed_time'] < iteration['elapsed_time']: 
        results['worst_metrics']['worst_time'] = dict(iteration)
        results['worst_metrics']['worst_time']['algoritm'] = algoritm

In [44]:
# Dicionário para armazenar resultados
results = {
    'iterations':0,
    'start_time':time.time(),
    'elapsed_time':0,
    'knn': {'iterations': [], 'best_accuracy': None, 'best_time': None, 'worst_accuracy': None, 'worst_time': None},
    'dt': {'iterations': [], 'best_accuracy': None, 'best_time': None, 'worst_accuracy': None, 'worst_time': None},
    'svm': {'iterations': [], 'best_accuracy': None, 'best_time': None, 'worst_accuracy': None, 'worst_time': None},
    'mlp': {'iterations': [], 'best_accuracy': None, 'best_time': None, 'worst_accuracy': None, 'worst_time': None},
    'best_metrics': {'best_accuracy': None, 'best_time': None},
    'worst_metrics': {'worst_accuracy': None, 'worst_time': None}
}

# Laço para treinar os modelos 10 vezes e armazenar as metricas
for i in range(10):
    
    ### KNN Classifier
    knn_iteration = {        
        'iteration':i,        
        'parameters':{
            'k':np.random.randint(1, 21), # Valor de K aleatório entre 1 e 20
            'weights': np.random.choice(['uniform', 'distance']) # Pesos aleatórios
        },
        'start_time': time.time()
    }
    knn_clf = KNeighborsClassifier(n_neighbors=knn_iteration['parameters']['k'], weights=knn_iteration['parameters']['weights'])
    update_results(results,'knn',knn_iteration,y_val,knn_clf)
    
    ### Decision Tree Classifier
    dt_iteration = {
        'iteration':i,        
        'parameters':{
            'max_depth': np.random.randint(1, 11),  # Aleatório entre 1 e 10
            'min_samples_split': np.random.randint(2, 11),  # Aleatório entre 2 e 10
            'criterion': np.random.choice(['gini', 'entropy'])  # Critério aleatório
        },
        'start_time': time.time()
    }
    dt_clf = DecisionTreeClassifier(max_depth=dt_iteration['parameters']['max_depth'], min_samples_split=dt_iteration['parameters']['min_samples_split'], criterion=dt_iteration['parameters']['criterion'])
    update_results(results,'dt',dt_iteration,y_val,dt_clf)

    ### SVM Classifier
    svm_iteration = {
        'iteration':i,        
        'parameters':{
            'C': np.random.uniform(0.1, 10.0),  # Valor de erro aleatório (C)
            'kernel': np.random.choice(['linear', 'poly', 'rbf'])  # Tipo de Kernel aleatório
        },
        'start_time': time.time()
    }
    svm_clf = SVC(C=svm_iteration['parameters']['C'], kernel=svm_iteration['parameters']['kernel'])
    update_results(results,'svm',svm_iteration,y_val,svm_clf)    

    ### MLP Classifier
    mlp_iteration = {
        'iteration':i,        
        'parameters':{
            'epochs': np.random.randint(50, 301),  # Número de épocas aleatório entre 50 e 300
            'learning_rate': np.random.uniform(0.001, 0.1),  # Taxa de aprendizagem aleatória
            'hidden_layers': np.random.randint(1, 6),  # Número de camadas escondidas aleatório entre 1 e 5
            'activation': np.random.choice(['identity', 'logistic', 'tanh', 'relu'])  # Função de ativação aleatória
        },
        'start_time': time.time()
    }
    mlp_clf = MLPClassifier(hidden_layer_sizes=(mlp_iteration['parameters']['hidden_layers'],), max_iter=mlp_iteration['parameters']['epochs'], learning_rate_init=mlp_iteration['parameters']['learning_rate'], activation=mlp_iteration['parameters']['activation'])
    update_results(results,'mlp',mlp_iteration,y_val,mlp_clf) 

    results['iterations'] += 1

results['elapsed_time'] = time.time() - results['start_time']
# Exibir resultados
print(json.dumps(results, indent=4))


{
    "iterations": 2,
    "start_time": 1729832717.4059594,
    "elapsed_time": 0.49092864990234375,
    "knn": {
        "iterations": [
            {
                "iteration": 0,
                "parameters": {
                    "k": 20,
                    "weights": "distance"
                },
                "start_time": 1729832717.4069088,
                "accuracy": 0.7775700934579439,
                "confusion_matrix": [
                    [
                        399,
                        5
                    ],
                    [
                        114,
                        17
                    ]
                ],
                "elapsed_time": 0.06830000877380371
            },
            {
                "iteration": 1,
                "parameters": {
                    "k": 7,
                    "weights": "uniform"
                },
                "start_time": 1729832717.6526887,
                "accuracy": 0.7813084112149533,
       